In [2]:

    print(f.readline())

# ---------------------------------- WARNING ----------------------------------------



In [117]:
import numpy as np

(2650,)


In [274]:
# for a in y:
#     print(a)

In [275]:
# for idx,x in enumerate(y):
#     print(idx,x)


In [276]:

z.shape

(2618, 5)

In [31]:
z

array([['USGS', '07374000', '2015-01-01', '345000', 'A\n'],
       ['USGS', '07374000', '2015-01-02', '353000', 'A\n'],
       ['USGS', '07374000', '2015-01-03', '364000', 'A\n'],
       ...,
       ['USGS', '07374000', '2022-03-01', '704000', 'P\n'],
       ['USGS', '07374000', '2022-03-02', '729000', 'P\n'],
       ['USGS', '07374000', '2022-03-03', '753000', 'P\n']], dtype='<U10')

In [277]:
a.shape

(1, 5)

In [292]:

b.shape

(2619, 5)

In [279]:
a.shape

(1, 5)

In [280]:
b[0]

array(['Gov Ent', 'Gage ID', 'Acq Date', 'cubic ft/s', 'unknown'],
      dtype='<U10')

In [293]:


b[-5:]

array([['USGS', '07374000', '2022-02-22', '572000', 'P\n'],
       ['USGS', '07374000', '2022-02-23', '567000', 'P\n'],
       ['USGS', '07374000', '2022-02-24', '571000', 'P\n'],
       ['USGS', '07374000', '2022-02-25', '589000', 'P\n'],
       ['USGS', '07374000', '2022-02-26', '615000', 'P\n']], dtype='<U10')

In [294]:

c

array(['345000', '353000', '364000', ..., '571000', '589000', '615000'],
      dtype='<U10')

error


(2614,)

In [296]:
print(len(c))

2614


In [297]:
c[-1]

'615000'

In [298]:
d[d<0]

array([-12345678.])

torch.Size([1, 1, 1, 3])

In [2]:
import torch
import numpy as np

# https://nwis.waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=07374000&referred_module=sw&period=&begin_date=2015-01-01&end_date=2022-03-03

y = []
with open('streamflowmiss.txt','r') as f:
    for x in f:
        y.append(x)
y = np.asarray(y)
print('number of lines in text file',y.shape)

streamflows = y[31:] #gets rid of meta data 
z = []
for idx,x in enumerate(streamflows):
    y = x.split('\t')
    z.append(y)

z = np.asarray(z)
a = np.expand_dims(np.asarray(['Gov Ent','Gage ID','Acq Date','cubic ft/s','unknown']),0)
b = np.vstack((a,z))
b = b[:-5] #gets rid of dates we don't want
print(b[1],'\n',b[-1])
c = b[1:,3]
d = np.empty((0))
for x in c:
    try:
        d = np.append(d,int(x))
    except:
        print('+1 empty, replaced with -12345678')
        d = np.append(d,-12345678)


x = torch.Tensor([.333,.333,.333]).float()
x = x.unsqueeze(0).unsqueeze(0).unsqueeze(0)
y = torch.from_numpy(d).float().unsqueeze(0).unsqueeze(0).unsqueeze(0)

kernel_height = 1
kernel_width = 3
unfold = torch.nn.Unfold(kernel_size = (kernel_height,kernel_width),stride=(kernel_height,kernel_width))

#using x, you can control what range of days you want to start from
#y is based off of d which is based off of b
#it's streamflow in cubic feet / second
X = 91 #91 in this case is just ninety first in list starting in 2015
arr_unfold = unfold(y[:,:,:,X:]).permute(2,0,1).reshape(-1,1,kernel_height,kernel_width)
streamflow_3day = torch.FloatTensor([torch.mean(x) for x in arr_unfold])

#time column for reference
c = np.asarray(b[1:,2])
streamtime = c.copy()
time_idx = torch.arange(X,c.shape[0]).unsqueeze(0).unsqueeze(0).unsqueeze(0).float()

time_idx_unfold = unfold(time_idx).permute(2,0,1).reshape(-1,1,kernel_height,kernel_width)
dates_of_acquisition = []
for idx,x in enumerate(time_idx_unfold):
    y = f'{streamtime[int(x[0,0,0])]}_{streamtime[int(x[0,0,2])]}'
#     print(y)
    dates_of_acquisition.append(y)
#     print(streamtime[int(x[0,0,0])],'_',streamtime[int(x[0,0,2])])

streamflow_ship = np.transpose(np.vstack((np.expand_dims(dates_of_acquisition,0),np.expand_dims(streamflow_3day,0))))
streamflow_ship_txt = [str(x) for x in streamflow_ship]
print(streamflow_ship_txt[0])
# for idx,x in enumerate(streamflow_ship):
#     print(idx,x)
    
txt_file = open("streamflow_3day_mississippi_for_neuralnet.txt","w")
for idx,x in enumerate(streamflow_ship_txt):
    txt_file.write(f'{idx} {x} \n')
txt_file.close()
print('CHECKPOINT!')

number of lines in text file (2650,)
['USGS' '07374000' '2015-01-01' '345000' 'A\n'] 
 ['USGS' '07374000' '2022-02-26' '615000' 'P\n']
+1 empty, replaced with -12345678
['2015-04-02_2015-04-04' '1020000.0']
CHECKPOINT!


In [29]:
streamz = np.expand_dims(np.float32(streamflow_ship[:,1]),1)

In [30]:
streamz.shape

(841, 1)

In [37]:
streamz_tensor = torch.from_numpy(streamz).unsqueeze(1).unsqueeze(1)
torch.save(streamz_tensor,'streamz.pt')

In [39]:
streamz_tensor.shape

torch.Size([841, 1, 1, 1])

In [40]:
streamz_tensor[0]

tensor([[[1020000.]]])